In [1]:
import time
import numpy as np
from datetime import datetime
from calendar import timegm
from pymongo import MongoClient, ASCENDING, DESCENDING, ReturnDocument
import pyvisa
import serial
from pprint import pprint

## Connect to Database

In [2]:
def dbConnect():
    client = MongoClient('mongodb://localhost/')
    collection = client.data.adr2datas
    infocollection = client.control.adr2controls
    jobcollection = client.jobs.adr2jobs
    return collection, infocollection, jobcollection
collection,infocollection, jobcollection = dbConnect()

## Connect to Instruments

In [3]:
def connectInstruments():
    rm = pyvisa.ResourceManager()
    rm.list_resources()
    inst1 = rm.open_resource('GPIB0::1::INSTR', read_termination = '\x00')
    inst2 = rm.open_resource('GPIB0::2::INSTR', read_termination = '\x00')
    inst3 = rm.open_resource('GPIB0::6::INSTR', read_termination = '\x00')
    return [inst1,inst2,inst3]

## Create Information and Data Entries (DO NOT RUN THESE)

In [4]:
def get_info():
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "fridgeStatus": "Warm",
        "currentJob": "None",
        "switchState": "Closed",
        "currentLimit": 9.2,
        "maxVoltage": 400,
        "voltageStep": 1,
        "command": "None",
        "jobStart": timegm(datetime.now().timetuple())
    }
    return post
get_info()

{'command': 'None',
 'currentJob': 'None',
 'currentLimit': 9.2,
 'fridgeStatus': 'Warm',
 'jobStart': 1448453383,
 'maxVoltage': 400,
 'switchState': 'Closed',
 'timeStamp': 1448453383,
 'voltageStep': 1}

In [9]:
command = 'Magup'
infocollection.find_one_and_update({},{'$set': {'command': command}})

{u'_id': ObjectId('564cbbf46545eb8ebc45f947'),
 u'command': u'Magup',
 u'currentJob': u'Magdown',
 u'currentLimit': 9.2,
 u'fridgeStatus': u'Magdown',
 u'jobStart': 1447953419,
 u'maxVoltage': 400,
 u'switchState': u'Closed',
 u'timeStamp': 1447851428,
 u'voltageStep': 1}

In [5]:
maxVoltage = 500
infocollection.find_one_and_update({},{'$set': {'maxVoltage': maxVoltage}})

{u'_id': ObjectId('564cbbf46545eb8ebc45f947'),
 u'command': u'None',
 u'currentJob': u'None',
 u'currentLimit': 9.2,
 u'fridgeStatus': u'Cold (Magged Down)',
 u'jobStart': 1447959912,
 u'maxVoltage': 500,
 u'switchState': u'Closed',
 u'timeStamp': 1447851428,
 u'voltageStep': 1}

## Get Data from Instruments

In [7]:
def get_data(inst):
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": float(inst[1].ask('KRDG? B')),
#         "oneKTemp": float(inst[0].ask('KRDG? C')),
        "oneKTemp": 0,
        "threeKTemp": float(inst[0].ask('KRDG? A')),
        "sixtyKTemp": float(inst[0].ask('KRDG? B')),
        "magnetVoltage": float(inst[0].ask('SRDG? D')),
        "psVoltage": float(inst[2].ask('MEAS:VOLT?')),
        "psCurrent": float(inst[2].ask('MEAS:CURR?')),
        "currentJob": "None",
        "percentComplete": round(float(inst[2].ask('MEAS:CURR?'))/9 * 100),
        "switchState": "Closed"
    }
    return post
get_data(connectInstruments())

{'baseTemp': 0.04,
 'currentJob': 'None',
 'magnetVoltage': -0.00156,
 'oneKTemp': 0,
 'percentComplete': -0.0,
 'psCurrent': -0.00446025,
 'psVoltage': -0.00941257,
 'sixtyKTemp': 49.039,
 'switchState': 'Closed',
 'threeKTemp': 2.8798,
 'timeStamp': 1449149664}

## Setup Log File

In [5]:
logFileName = 'ADR2Log.txt'
inst = connectInstruments()
ps = inst[2]

## Main Loop

In [ ]:
currentJob = None
statusString = ""
while True:
    # Open Log File
    logFile = open(logFileName, 'a')

    # Read Data
    try:
        data = get_data(inst)

    # Log errors if they occur and reconnect
    except UnicodeDecodeError:
        string = "Read error at " + datetime.now().strftime('%H:%M:%S') + '\n'
        print(string)
        logFile.write(string)
        inst = connectInstruments()

    # Insert Data into Database
    else:
        if (data['sixtyKTemp'] <= 300):
            try:
                collection.insert(data)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))

    # Check for impending jobs
#     nextJob = None
#     for job in jobcollection.find({}).limit(1).sort('startTime', ASCENDING):
#         nextJob = job
#     if (nextJob):
#         print("Next Job at " + nextJob['timeString'])
    
#     if (nextJob and (nextJob['startTime'] - datetime.utcnow()).seconds < 5):
#         # Mark previous current job as completed
#         if (currentJob and currentJob['inProgress']):
#             jobcollection.find_one_and_update({'_id': currentJob['_id']}, {'$set': {'inProgress': False, 'completed': True}})
        
#         # Next Job becomes current job
#         currentJob = nextJob
#         print("Starting Scheduled " + currentJob['type'])
#         jobcollection.find_one_and_update({'_id': currentJob['_id']}, {'$set': {'inProgress': True}})
# #         info = infocollection.find_one_and_update({},{'$set': {'command': currentJob['type']}})
    
    # Get information from Database

    info = infocollection.find().next()

    if info['command'] == 'Magup':
        #Ensure fridge is actually cold enough to mag up!
        if data['threeKTemp'] > 4.5:
            pass
        elif info['currentJob'] != 'Magup':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magup', 'jobStart': data['timeStamp']}})
            logString = "Start Magup to " + str(info['maxVoltage']) + "mV at " + datetime.now().strftime('%a, %c')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Magdown':
        if info['currentJob'] != 'Magdown':
            if info['currentJob'] == 'Soak':
                stopSoakTime = datetime.now()
                t = (stopSoakTime - startSoakTime)
                logString = "Soaked for {} days, {}h: {}m: {}s".format(t.days,t.seconds//3600,(t.seconds//60)%60, t.seconds%60)
                print(logString)
                logFile.writelines([logString, '\n'])
            
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magdown', 'jobStart': data['timeStamp']}})
            logString = "Start Magdown at " + datetime.now().strftime('%a, %c')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Soak':
        if info['currentJob'] != 'Soak':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Soak', 'jobStart': data['timeStamp']}})
            logString = "Start Soak at " + datetime.now().strftime('%a, %c')
            startSoakTime = datetime.now()
            print(logString)
            logFile.writelines([logString, '\n'])

    else:
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'None'}})
        if data['baseTemp'] < 3 and data['baseTemp'] != 0.0:
            statusString = "Cold (Magged Down)"
        elif data['threeKTemp'] < 4.5:
            statusString = "Cold (3K)"
        elif (data['sixtyKTemp'] < data['threeKTemp']) and (data['sixtyKTemp'] > 50):
            statusString = "Cooling Down"
        elif (data['threeKTemp'] > 285):
            statusString = "Warm"
        elif (data['sixtyKTemp'] > data['threeKTemp']) and (data['sixtyKTemp'] > 60):
            statusString = "Warming Up"
        else:
            statusString = "Uncertain State"
        
    

    # Control Magnet
    job = info['currentJob']

    if job == 'Magup':
        # Increase Magnet Voltage
        currentV = float(ps.ask('VOLT?'))*1000
        if currentV < info['maxVoltage']:
            nextV = currentV + info['voltageStep']
            ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
            print(str(nextV) + "mV\t" + "Current: " + ps.ask('MEAS:CURR?') + "\tVoltage: " + ps.ask('MEAS:VOLT?'))
        else:
            print('Max Voltage Reached')
            if data['psCurrent'] > 9.0:
                print('9A reached')
                info = infocollection.find_one_and_update({},{'$set': {'command': 'Soak'}})
        statusString = "Magup"
    elif job == 'Magdown':
        # Decrease Magnet Voltage
        currentV = float(ps.ask('VOLT?'))*1000
        if currentV > 0:
            nextV = currentV - info['voltageStep']
            ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
            print(str(nextV) + "mV\t" + "Current: " + ps.ask('MEAS:CURR?') + "\tVoltage: " + ps.ask('MEAS:VOLT?'))
        else:
            print('Zero Voltage Reached')
            info = infocollection.find_one_and_update({},{'$set': {'command': 'None'}})
        statusString = "Magdown"
    elif job == 'Soak':
        # Soak
        statusString = "Soak"
    else:
        pass
    
    info = infocollection.find_one_and_update({},{'$set': {'fridgeStatus': statusString}})
    # Close Log File
    logFile.close()

    # Wait
    time.sleep(5)